In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(32, 16, kernel_size=1)
        self.bn4   = nn.BatchNorm2d(16)

        self.conv5 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn5   = nn.BatchNorm2d(32)

        self.conv6 = nn.Conv2d(32, 16, kernel_size=1)
        self.bn6   = nn.BatchNorm2d(16)

        self.conv7 = nn.Conv2d(16, 16, kernel_size=3)
        self.bn7   = nn.BatchNorm2d(16)

        # Fully connected layer
        self.fc1 = nn.Linear(400, 10)

        # Dropout layers
        self.dropout1 = nn.Dropout(0.1)   # 10% dropout


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)   # pooling after activation

        x = F.relu(self.conv4(x))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(self.conv7(x)))


        x = x.view(-1, 400)   # flatten

        x = self.dropout1(x)
        x = self.fc1(x)

        return F.log_softmax(x, dim=1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
            Conv2d-5           [-1, 32, 28, 28]           4,640
       BatchNorm2d-6           [-1, 32, 28, 28]              64
            Conv2d-7           [-1, 16, 14, 14]             528
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
           Conv2d-10             [-1, 16, 7, 7]             528
      BatchNorm2d-11             [-1, 16, 7, 7]              32
           Conv2d-12             [-1, 16, 5, 5]           2,320
      BatchNorm2d-13             [-1, 16, 5, 5]              32
          Dropout-14                  [

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-10., 10.), fill=0),
                        transforms.RandomAffine(0, translate=(0.1, 0.1)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.49MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 161kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.30MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.71MB/s]


In [5]:
print(len(test_loader))
print(len(train_loader))

79
469


In [6]:
# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

In [7]:
from tqdm import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train(model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    processed = 0


    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        output = model(data)
        # loss = F.nll_loss(output, target)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        correct += GetCorrectPredCount(output, target)
        processed += len(data)
        pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

    train_acc.append(100*correct/processed)
    train_losses.append(train_loss/len(train_loader))

def test(model, device, test_loader,criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.002)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.25)
criterion = nn.CrossEntropyLoss()

for epoch in range(1, 20):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, epoch,criterion)
    test(model, device, test_loader,criterion)
    scheduler.step()

Epoch 1


Train: Loss=0.0774 Batch_id=468 Accuracy=92.74: 100%|██████████| 469/469 [02:29<00:00,  3.14it/s]



Test set: Average loss: 0.0004, Accuracy: 9842/10000 (98.42%)

Epoch 2


Train: Loss=0.0727 Batch_id=468 Accuracy=97.36: 100%|██████████| 469/469 [02:28<00:00,  3.17it/s]



Test set: Average loss: 0.0003, Accuracy: 9871/10000 (98.71%)

Epoch 3


...


Test set: Average loss: 0.0001, Accuracy: 9954/10000 (99.54%)

Epoch 7


Train: Loss=0.0310 Batch_id=468 Accuracy=98.86: 100%|██████████| 469/469 [02:27<00:00,  3.18it/s]



Test set: Average loss: 0.0001, Accuracy: 9941/10000 (99.41%)

Epoch 8


Train: Loss=0.0289 Batch_id=121 Accuracy=98.96:  26%|██▌       | 122/469 [00:38<01:49,  3.16it/s]


KeyboardInterrupt: 